In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os
import time

In [2]:
rech = 'https://rech.com/busca?q='
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [3]:
query = pd.read_csv('produtos_bot_yury_not_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [4]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [5]:
def findOEM(text):
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) > 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [6]:
def rechSearch(nav, cod, index):
    nav.get(str(rech + cod))
    try:
        elemento = nav.find_element(By.ID, 'showcase')
        elementos = elemento.find_elements(By.CLASS_NAME, 'y-item-wrapper-showcase')
        urls = []
        for elemento in elementos:
            url = elemento.find_element(By.TAG_NAME, 'link').get_attribute('href')
            urls.append(url)
        for url in urls:
            src = []
            keys = []
            values = []
            nav.get(str(url))
            nav.implicitly_wait(0.8)
            elementos = nav.find_elements(By.CLASS_NAME, 'zoomImg')
            for elemento in elementos:
                src.append(elemento.get_attribute('src'))
            keys.append('Nome:')
            values.append(nav.find_element(By.CLASS_NAME, 'product__name').text)
            keys.append('Descrição:')
            values.append(nav.find_element(By.CLASS_NAME, 'panel-description').text)
            elemento = nav.find_element(By.CLASS_NAME, 'table-specifications')
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr th')
            for k in elementos:
                keys.append(k.text)
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr td')
            for v in elementos:
                values.append(v.text)
            keys.append('URL: ')
            values.append(url)
            dict_infos = dict(zip(keys, values))
            for x in dict_infos.keys():
                y = dict_infos.get(x)
                y = y.replace('\n', ', ')
                dict_infos.update({x : y})
            try:
                temp = dict_infos['Aplicação Equipamentos'].split(' (')
                equips = []
                for item in temp:
                    item = item.replace('1 ATÉ 999999999) -', '')
                    item = item.replace(', VER MAIS', '')
                    if item:
                        item = item.strip()
                        equips.append(item)
                dict_infos.update({'Aplicação Equipamentos': equips})
            except Exception as e:
                print(e)
            try:
                temp = dict_infos['Código da Montadora'].split(',')
                cods = []
                for item in temp:
                    cods.append(item.strip())
                dict_infos.update({'Código da Montadora': cods})
            except Exception as e:
                print(e)
            images = {str(index) : src}
            dict_infos = {str(index) : dict_infos}
            dict_infos = str(dict_infos)
            dict_infos = dict_infos.replace("'", '"')
            images = str(images)
            images = images.replace("'", '"')
            with open(f'{cwd}/Rech/prods.json', 'a') as teste:
                teste.write(dict_infos)
            with open(f'{cwd}/Rech/images.json', 'a') as teste:
                teste.write(images)
            
    except Exception as e:
        notFound.loc[len(notFound)] = [index, 'Não encontrado no site']
        return

In [7]:
def loginRech(nav):
    nav.get('https://www.rech.com/login?targetUri=%2F')
    elemento = nav.find_element(By.ID, 'username')
    elemento.send_keys('yuri@cotripal.com.br')
    elemento = nav.find_element(By.ID, 'password')
    elemento.send_keys('Testes1!')
    elemento = nav.find_element(By.ID, 'submit')
    elemento.click()
    nav.implicitly_wait(0.8)

In [8]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)
# produtos = dict(zip(query['ID'], query['Produto']))

loginRech(nav)

In [9]:
# rechSearch(nav, '87041572', 1)
#https://rech.com/busca?q=R110126/187

In [10]:
# rechSearch(nav, 'R110126/187', 1)

In [11]:
time.sleep(1)
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findOEM(cod)
    for item in cods:
        rechSearch(nav, item, ID)

'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Aplicação Equipamentos'
'Aplicação Equipamentos'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplicação Equipamentos'
'Código da Montadora'
'Aplica

In [12]:
notFound = notFound.drop_duplicates()
notFound.to_excel(f'{cwd}/Rech/notFound.xlsx')